# Big Data Modelling and Management - Queries

**Group 24 members:** <br>
Filipe Lourenço (R20170799), <br>
Guilherme Neves (R20170749), <br>
Rui Monteiro (R20170796), <br>
Vasco Pestana (R20170803)

**MSc:** Data Science and Advanced Analytics - Nova IMS <br>
2020/2021

In [12]:
from pymongo import MongoClient, DESCENDING, ASCENDING, TEXT, GEO2D
import mysql.connector
from bson.objectid import ObjectId
from bson.decimal128 import Decimal128
import datetime
from decimal import *
from urllib.request import urlopen
import json
import csv
from copy import deepcopy
import pandas as pd
from bson.son import SON

### Connecting to the MongoDB database

In [13]:
host="rhea.isegi.unl.pt"
port="27041"
user="GROUP_24"
password="T0b5Sez8prB8jo9ohdb907GL5U0S24GX"
protocol="mongodb"
client = MongoClient(f"{protocol}://{user}:{password}@{host}:{port}")
db = client.WWI

## Web Team
1. From which states are our suppliers from?

In [5]:
lookup_query = {
        "$lookup":{
           "from": "Countries_StateProvinces_Cities",
           "localField": "DeliveryCityID",
           "foreignField": "CityID",
           "as": "delivery_city"
        }
    }

group_query = { 
        "$group" : {
            "_id" : {"state":"$delivery_city.state.StateProvinceName"},
            "number_suppliers": {"$sum": "$SupplierID"}
        } 
    }

sort_query = { 
        "$sort" : {
            "number_suppliers" : -1
        } 
    }
    

pipeline = [lookup_query, group_query, sort_query]

r = db.Purchasing_Suppliers.aggregate(pipeline)

Result_Q1 = list(r)

Result_Q1

[{'_id': {'state': [['California']]}, 'number_suppliers': 23},
 {'_id': {'state': [['Tennessee']]}, 'number_suppliers': 14},
 {'_id': {'state': [['Minnesota']]}, 'number_suppliers': 12},
 {'_id': {'state': [['South Dakota']]}, 'number_suppliers': 11},
 {'_id': {'state': [['New Jersey']]}, 'number_suppliers': 10},
 {'_id': {'state': [['North Carolina']]}, 'number_suppliers': 9},
 {'_id': {'state': [['Missouri']]}, 'number_suppliers': 5},
 {'_id': {'state': [['Kentucky']]}, 'number_suppliers': 4},
 {'_id': {'state': [['Washington']]}, 'number_suppliers': 2},
 {'_id': {'state': [['Indiana']]}, 'number_suppliers': 1}]

2. From which state province are the customers who have a higher credit limit?

In [6]:
lookup_query = {
        "$lookup":{
           "from": "Countries_StateProvinces_Cities",
           "localField": "DeliveryCityID",
           "foreignField": "CityID",
           "as": "delivery_city"
        }
    }

group_query = { 
        "$group" : {
            "_id" : {"state":"$delivery_city.state.StateProvinceName"},
            "credit_limit_avg": {"$avg": "$CreditLimit"}
        } 
    }

sort_query = { 
        "$sort" : {
            "credit_limit_avg" : -1
        } 
    }

limit_query = { 
        "$limit" : 1
    } 
    
pipeline = [lookup_query, group_query, sort_query, limit_query]

r = db.Sales_Customers.aggregate(pipeline)

Result_Q2 = list(r)

Result_Q2

[{'_id': {'state': [['Tennessee']]},
  'credit_limit_avg': Decimal128('3157.50')}]

## Warehouse Group
3. To know which items get ordered together the most?

In [48]:
match_query_1 = {
    '$match': {
        'Items_Description': { "$exists": True }
    }
}

project_query = {
    '$project': {
        'Items_Description': 1,
        'array_size': { "$size": "$Items_Description" }
    }
}

match_query_2 = {
    '$match': {
        'array_size': { "$gt": 1 }
    }
}

groupby_query = {
    '$group':{
        '_id': {'Items_Description': '$Items_Description'},
        'count': {'$sum': 1}
    }
}

sort_query = {
    '$sort': {'count': -1}
}

limit_query = {
    '$limit': 15
}

pipeline = [match_query_1, project_query, match_query_2, groupby_query, sort_query, limit_query]

r = db.All_Sales_Orders.aggregate(pipeline)

Result_Q3 = list(r)

Result_Q3

[{'_id': {'Items_Description': ['10 mm Double sided bubble wrap 20m',
    'Large  replacement blades 18mm']},
  'count': 2},
 {'_id': {'Items_Description': ['32 mm Anti static bubble wrap (Blue) 20m',
    'Developer joke mug - a foo walks into a bar (White)']},
  'count': 2},
 {'_id': {'Items_Description': ['Black and orange glass with care despatch tape  48mmx100m',
    'Ride on big wheel monster truck (Black) 1/12 scale']},
  'count': 2},
 {'_id': {'Items_Description': ['Dinosaur battery-powered slippers (Green) M',
    'Large sized bubblewrap roll 50m',
    '"The Gu" red shirt XML tag t-shirt (White) S']},
  'count': 2},
 {'_id': {'Items_Description': ['20 mm Anti static bubble wrap (Blue) 10m',
    'Clear packaging tape 48mmx100m']},
  'count': 2},
 {'_id': {'Items_Description': ['USB food flash drive - fortune cookie',
    'Ride on toy sedan car (Pink) 1/12 scale']},
  'count': 2},
 {'_id': {'Items_Description': ['USB missile launcher (Green)',
    'Red and white urgent  heavy des

4. Which items get ordered the most in bulk (bigger amounts)?

In [22]:
unwind_query = {
    "$unwind": "$stock_item"
}

match_query = {
    '$match': {'stock_item.Quantity': {'$gt':0}}
}

groupby_query = {
    '$group':{
        '_id':{'StockItemName':'$StockItemName'},
        'Count': {'$sum': 1},
        'Average_Quantity': {'$avg':'$stock_item.Quantity'}}
}

sort_query = {
    '$sort': {'Count':-1, 'Average_Quantity':-1}
}

project_query = {
    '$project': {
        "Rounded_Avg_Quantity": {"$round": ["$Average_Quantity", 0] }
    }
}

limit_query = {
    '$limit': 3
}

pipeline = [unwind_query, match_query, groupby_query, sort_query, project_query, limit_query]

r = db.Sales_OrderLines_Warehouse_StockItems.aggregate(pipeline)

Result_Q4 = list(r)

Result_Q4

[{'_id': {'StockItemName': '"The Gu" red shirt XML tag t-shirt (White) XXL'},
  'Rounded_Avg_Quantity': 68.0},
 {'_id': {'StockItemName': 'Developer joke mug - this code was generated by a tool (White)'},
  'Rounded_Avg_Quantity': 6.0},
 {'_id': {'StockItemName': '"The Gu" red shirt XML tag t-shirt (White) S'},
  'Rounded_Avg_Quantity': 70.0}]

5. Which customers have delivery addresses under 10km of distance to the customer with customerID 961?

In [10]:
# Query the coordinates of the client with the CustomerID equal to 961
coords_client_961 = list(db.Sales_Customers.find({'CustomerID': 961}))[0]['DeliveryLocationCoordinates']

# 1 km equals 111.12 degrees values, so the max distance of 10km must be divided by 111.12 to obtain the maximum distance
# between customers in degrees.
geo_query = {"DeliveryLocationCoordinates": SON([("$near", coords_client_961), ("$maxDistance", 10/111.12)])}

project_query = {'CustomerName':1, 'DeliveryPostalCode':1, 'PostalAddressLine2':1, 'DeliveryLocationCoordinates':1}

Result_Q5 = list(db.Sales_Customers.find(geo_query, project_query))

Result_Q5

[{'_id': ObjectId('60b17444b18bb0a6dfaf7369'),
  'CustomerName': 'Rani Ganguly',
  'DeliveryPostalCode': '90567',
  'PostalAddressLine2': 'Benjareeville',
  'DeliveryLocationCoordinates': [34.23, -117.263]},
 {'_id': ObjectId('60b17443b18bb0a6dfaf72fc'),
  'CustomerName': 'Dominic Davignon',
  'DeliveryPostalCode': '90192',
  'PostalAddressLine2': 'Glissenville',
  'DeliveryLocationCoordinates': [34.2389, -117.234]},
 {'_id': ObjectId('60b1743db18bb0a6dfaf71a8'),
  'CustomerName': 'Malorie Bousquet',
  'DeliveryPostalCode': '90005',
  'PostalAddressLine2': 'Biteville',
  'DeliveryLocationCoordinates': [34.235, -117.308]}]

## CFO
6. Would like to know the monthly order count?

In [18]:
project_query = {
    "$project": {
        "order_year": { "$year": "$OrderDate" },
        "order_month": { "$month": "$OrderDate" }
    }
}

groupby_query = {
    '$group': {
        '_id': {"order_year":"$order_year", 
                "order_month":"$order_month"},
        'count' : {'$sum' : 1}
    }
}

sort_query = {
    '$sort': {
        'count': -1
    }
}

pipeline = [project_query, groupby_query, sort_query]

r = db.All_Sales_Orders.aggregate(pipeline)

Result_Q6 = list(r)

Result_Q6

[{'_id': {'order_year': 2015, 'order_month': 7}, 'count': 2273},
 {'_id': {'order_year': 2015, 'order_month': 4}, 'count': 2111},
 {'_id': {'order_year': 2015, 'order_month': 9}, 'count': 2065},
 {'_id': {'order_year': 2016, 'order_month': 5}, 'count': 2047},
 {'_id': {'order_year': 2014, 'order_month': 7}, 'count': 2008},
 {'_id': {'order_year': 2015, 'order_month': 6}, 'count': 1989},
 {'_id': {'order_year': 2016, 'order_month': 3}, 'count': 1969},
 {'_id': {'order_year': 2015, 'order_month': 10}, 'count': 1965},
 {'_id': {'order_year': 2015, 'order_month': 5}, 'count': 1959},
 {'_id': {'order_year': 2015, 'order_month': 12}, 'count': 1951},
 {'_id': {'order_year': 2016, 'order_month': 4}, 'count': 1951},
 {'_id': {'order_year': 2016, 'order_month': 1}, 'count': 1944},
 {'_id': {'order_year': 2014, 'order_month': 10}, 'count': 1921},
 {'_id': {'order_year': 2014, 'order_month': 12}, 'count': 1914},
 {'_id': {'order_year': 2014, 'order_month': 5}, 'count': 1908},
 {'_id': {'order_year

7. Would like to know the average monthly sales prices?

In [83]:
project_query_1 = {
    "$project": {
        "OrderDate": 1,
        "order_lines.Quantity": 1,
        "order_lines.UnitPrice": 1 
    }
}

unwind_query = {
    "$unwind": "$order_lines"
    }

project_query_2 = {
    "$project": {
        "order_year": { "$year": "$OrderDate" },
        "order_month": { "$month": "$OrderDate" },
        "mult": {"$multiply": ["$order_lines.Quantity", "$order_lines.UnitPrice"]}
    }
}

groupby_query = {
    '$group': {
        '_id': {"order_year":"$order_year", 
                "order_month":"$order_month"},
        'avg_prices' : {'$avg': "$mult"}
    }
}

sort_query = {
    '$sort': {
        'avg_prices': -1
    }
}

project_query_3 = {
    '$project': {
        "rounded_avg_prices": {"$round": ["$avg_prices", 2] }
    }
}

pipeline = [project_query_1, unwind_query, project_query_2, groupby_query, sort_query, project_query_3]

r = db.All_Sales_Orders.aggregate(pipeline)

Result_Q7 = list(r)

Result_Q7

[{'_id': {'order_year': 2013, 'order_month': 5},
  'rounded_avg_prices': Decimal128('800.64')},
 {'_id': {'order_year': 2013, 'order_month': 4},
  'rounded_avg_prices': Decimal128('782.03')},
 {'_id': {'order_year': 2014, 'order_month': 3},
  'rounded_avg_prices': Decimal128('780.69')},
 {'_id': {'order_year': 2013, 'order_month': 6},
  'rounded_avg_prices': Decimal128('777.46')},
 {'_id': {'order_year': 2013, 'order_month': 11},
  'rounded_avg_prices': Decimal128('770.59')},
 {'_id': {'order_year': 2013, 'order_month': 9},
  'rounded_avg_prices': Decimal128('767.99')},
 {'_id': {'order_year': 2013, 'order_month': 7},
  'rounded_avg_prices': Decimal128('763.69')},
 {'_id': {'order_year': 2013, 'order_month': 2},
  'rounded_avg_prices': Decimal128('757.19')},
 {'_id': {'order_year': 2013, 'order_month': 12},
  'rounded_avg_prices': Decimal128('750.73')},
 {'_id': {'order_year': 2013, 'order_month': 10},
  'rounded_avg_prices': Decimal128('750.31')},
 {'_id': {'order_year': 2013, 'order_

8. Would like to know the yearly expenditures with suppliers (per supplier name)?

In [100]:
project_query = {
    "$project": {
        "transaction_year": { "$year": "$TransactionDate" },
        "SupplierName": "$purchasing_suppliers_categories_transactions.SupplierName",
        "TransactionAmount": 1
    }
}

groupby_query = {
    '$group': {
        '_id': {"transaction_year":"$transaction_year", 
                "SupplierName":"$SupplierName"},
        'sum_amount' : {'$sum': "$TransactionAmount"}
    }
}

sort_query = {
    '$sort': {
        '_id.transaction_year': -1
    }
}

pipeline = [project_query, groupby_query, sort_query]

r = db.All_Purchasing_Supplier_Categories_Transactions.aggregate(pipeline)

Result_Q8 = list(r)

Result_Q8

[{'_id': {'transaction_year': 2016, 'SupplierName': ['Fabrikam, Inc.']},
  'sum_amount': Decimal128('-2961969.90')},
 {'_id': {'transaction_year': 2016, 'SupplierName': ['A Datum Corporation']},
  'sum_amount': Decimal128('0.00')},
 {'_id': {'transaction_year': 2016, 'SupplierName': ['Litware, Inc.']},
  'sum_amount': Decimal128('-1310231.11')},
 {'_id': {'transaction_year': 2015, 'SupplierName': ['Litware, Inc.']},
  'sum_amount': Decimal128('1353734.69')},
 {'_id': {'transaction_year': 2015, 'SupplierName': ['Fabrikam, Inc.']},
  'sum_amount': Decimal128('2837528.40')},
 {'_id': {'transaction_year': 2014, 'SupplierName': ['Litware, Inc.']},
  'sum_amount': Decimal128('686937.21')},
 {'_id': {'transaction_year': 2014, 'SupplierName': ['Fabrikam, Inc.']},
  'sum_amount': Decimal128('1510955.10')},
 {'_id': {'transaction_year': 2013, 'SupplierName': ['Litware, Inc.']},
  'sum_amount': Decimal128('119343.32')},
 {'_id': {'transaction_year': 2013, 'SupplierName': ['Contoso, Ltd.']},
  'su

## Partnerships
9. Would like to know what’s the most common payment type?

In [56]:
lookup_query = {
    "$lookup": {
        "from": "Purchasing_SupplierTransactions_PaymentMethod",
        "localField": "payment_method.PaymentMethodID",
        "foreignField": "PaymentMethodID",
        "as": "Payment_Method_info"
    }
}

match_query = {
    '$match': {
        'payment_method.PaymentMethodID': { "$exists": True }
    }
}

unwind_query = {
    "$unwind": "$payment_method"
    }

groupby_query = {
    '$group': {
        '_id': {'Payment_Method':'$payment_method.Payment_Method_Name'},
        'count' : {'$sum' : 1}
    }
}

sort_query = {
    '$sort': {
        'count': -1
    }
}

limit_query = {
    '$limit': 1
}

pipeline = [lookup_query, match_query, groupby_query, limit_query]

r = db.Sales_CustomerTransactions_PaymentMethod.aggregate(pipeline)

Result_Q9 = list(r)

Result_Q9

[{'_id': {'Payment_Method': []}, 'count': 51959}]

10. Which supplier of Novelty Goods Supplier as the most transactions?

In [19]:
match_query = {
     '$match': {
         'purchasing_suppliers_categories_transactions.purchasing_suppliers_categories.SupplierCategoryName': { "$eq": 'Novelty Goods Supplier' }
     }
 }

unwind_query_1 = {
    "$unwind": "$purchasing_suppliers_categories_transactions"
    }

unwind_query_2 = {
    "$unwind": "$purchasing_suppliers_categories_transactions.purchasing_suppliers_categories"
    }

project_query = {
    "$project": {
        "SupplierCategoryName": "$purchasing_suppliers_categories_transactions.purchasing_suppliers_categories.SupplierCategoryName",
        "Supplier_Name": "$purchasing_suppliers_categories_transactions.SupplierName"
    }
}

groupby_query = {
    '$group': {
        '_id': {"Supplier_Category_Name":"$Supplier_Category_Name", 
                "Supplier_Name":"$Supplier_Name"},
        'count' : {'$sum' : 1}
    }
}

sort_query = {
    '$sort': {
        'count': -1,
        'Supplier_Name': -1
    }
}

limit_query = {
    '$limit': 1
}

pipeline = [match_query, unwind_query_1, unwind_query_2, project_query, groupby_query, sort_query, limit_query]

r = db.All_Purchasing_Supplier_Categories_Transactions.aggregate(pipeline)

Result_Q10 = list(r)

Result_Q10

[{'_id': {'Supplier_Name': 'Graphic Design Institute'}, 'count': 16}]

## Marketing Team
11. Want to make an appreciation post and needs the name of the sales person with the most invoices in 2013 (person who’s customers brought the most money)?

In [59]:
unwind_query = {
    "$unwind": "$invoice_lines"
    }

project_query_1 = {
    "$project": {
        "SalespersonPersonID":1,
        "invoice_year": { "$year": "$InvoiceDate" },
        "sales_amount": {"$multiply": ["$invoice_lines.UnitPrice", "$invoice_lines.Quantity"]}     
    }
}

match_query = {
    "$match": {
        "invoice_year": {"$eq": 2013}
    }
}

group_query = { 
        "$group" : {
            "_id" : {"salespersonid":"$SalespersonPersonID"},
            "number_invoices": {"$sum": 1},
            "total_sales_amount": {"$sum": "$sales_amount"}
        } 
    }

lookup_query = {
        "$lookup":{
           "from": "Application_People",
           "localField": "_id.salespersonid",
           "foreignField": "PersonID",
           "as": "app_sales_person"
        }
    }

project_query_2 = {
    "$project":{
        "_id": 0,
        "full_name":"$app_sales_person.FullName",
        "number_invoices":1,
        "total_sales_amount":1
    }
}

sort_query = { 
        "$sort" : {
            "number_invoices" : -1
        } 
    }


limit_query = { 
        "$limit" : 1
    } 

pipeline = [unwind_query, project_query_1, match_query, group_query, lookup_query, project_query_2, sort_query, limit_query]  

r = db.All_Sales_Invoices.aggregate(pipeline)

Result_Q11 = list(r)

Result_Q11

[{'number_invoices': 3148,
  'total_sales_amount': Decimal128('2545902.55'),
  'full_name': ['Hudson Onslow']}]